In [ ]:
# switch to tensors

In [1]:
import os
print(os.getcwd())
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

/home/sepidehparhami/tutoring/Research Project/yield-prediction


In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv3D, Flatten

In [3]:
yields = pd.read_csv(r'../yield.txt', delim_whitespace=True, index_col=0)

In [4]:
ligands = list(yields.index)

In [5]:
sample_file = pd.read_csv('../data/ESP allPoints_L01-DPPF.txt_interpolated.txt')
sample_file.columns = ('x', 'y', 'z', 'esp')

In [7]:
print(len(np.unique(sample_file['x'])),len(np.unique(sample_file['y'])),len(np.unique(sample_file['z'])))

30 30 30


In [31]:
np.max(sample_file['x'])

14.0

In [18]:
sample_file.shape

(26999, 4)

In [6]:
n_molecules = yields.shape[0]
n_coords = sample_file.shape[0]
dim_x, dim_y, dim_z = (len(np.unique(sample_file[data_axis])) for data_axis in ['x','y','z'])

In [7]:
# initialize a data frame and fill in ESPs from file

data = pd.DataFrame(index = ligands, columns=range(n_coords+1))

In [8]:
directory = '../data' 
      
for i,filename in enumerate(os.listdir(directory)):
    # print(i+1,'of',n_molecules)

    data_point = pd.read_csv('../data/{}'.format(filename))
    data_point.columns = ('x', 'y', 'z', 'esp')
    
    # print('number coords in file:',data_point.shape[0])

    #extract the ligand name from the filename
    name_parts = filename.split("_")
    if name_parts[1][0] == "T":
        ligand = name_parts[1][:4]
    else:
        ligand = name_parts[1][:3]
        
    #add label and data point to their respective arrays 
    data.loc[ligand] = data_point['esp']

In [9]:
data[n_coords] = np.zeros(n_molecules,dtype=type(data.iloc[0,0]))

In [17]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,26990,26991,26992,26993,26994,26995,26996,26997,26998,26999
L01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
L02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
L04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
L05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
L06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# turn data frame into 5-dimensional array (n_molecules,x,y,z,1 (size of ESP value)) array for keras input 
data_arr_5d = np.empty((n_molecules,dim_x,dim_y,dim_z,1),dtype=np.float32)
for i,l in enumerate(ligands):
    data_arr_5d[i] = np.reshape(data.loc[l].values, (dim_x,dim_y,dim_z,1))
# print(data_arr_5d[0])
print(data_arr_5d.shape)

(39, 30, 30, 30, 1)


In [35]:
sample_file.iloc[-1]

x      13.0
y      14.0
z      14.0
esp     0.0
Name: 26998, dtype: float64

In [13]:
model = Sequential()
model.add(Conv3D(n_molecules,kernel_size=10,activation='relu',input_shape=(dim_x,dim_y,dim_z,1)))
model.add(Flatten())
model.add(Dense(3))

In [46]:
model.compile(optimizer='adam',loss='MSE',metrics='MSE')

In [11]:
yields_arr = yields.values
yields.values[0:2]

array([[83.3,  8.9,  7.9],
       [70.8, 18.4, 10.8]])

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_arr_5d, yields_arr, test_size=0.2)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(31, 30, 30, 30, 1) (31, 3) (8, 30, 30, 30, 1) (8, 3)


In [52]:
model.fit(X_train,
          y_train,
          validation_split=0.2,
          epochs=3)

Epoch 1/3
1/1 [==============================] - 2s 2s/step - loss: 1672.8107 - val_loss: 1787.8073
Epoch 2/3
1/1 [==============================] - 2s 2s/step - loss: 1672.8109 - val_loss: 1787.8073
Epoch 3/3
1/1 [==============================] - 3s 3s/step - loss: 1672.8107 - val_loss: 1787.8073


In [54]:
model.evaluate(X_train,y_train)

1/1 [==============================] - 1s 628ms/step - loss: 1698.7776


1698.777587890625

In [55]:
model.evaluate(X_test,y_test)

1/1 [==============================] - 0s 288ms/step - loss: 2328.9004


2328.900390625

In [13]:
# keras tuner walkthrough https://www.tensorflow.org/tutorials/keras/keras_tuner
import kerastuner as kt
import tensorflow as tf

In [19]:
def build_model(hp):
    model = Sequential()
    model.add(Conv3D(hp.Int("units", min_value=32, max_value=2048, step=32),
                     hp.Int("kernel_size", min_value=2, max_value=6, step=1),
                     activation='relu',
                     input_shape=(dim_x,dim_y,dim_z,1)))
    model.add(Flatten())
    model.add(Dense(3))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate", values=[5e-2, 1e-2, 5e-3, 1e-3, 5e-4, 1e-4])),
        loss='MSE',
        metrics='MSE')
    return model

In [20]:
tuner = kt.BayesianOptimization(
    build_model,
    objective='val_loss',
    max_trials=5,
    overwrite=True)

In [21]:
tuner.search(X_train, y_train, epochs=3, validation_split=0.2)

Epoch 1/3
1/1 [==============================] - ETA: 0s - loss: 180071.5000 - MSE: 180071.500 - 6s 6s/step - loss: 180071.5000 - MSE: 180071.5000 - val_loss: 199774289920.0000 - val_MSE: 199774289920.0000
Epoch 2/3
1/1 [==============================] - ETA: 0s - loss: 146757086347264.0000 - MSE: 146757086347264.000 - 3s 3s/step - loss: 146757086347264.0000 - MSE: 146757086347264.0000 - val_loss: 4541380096.0000 - val_MSE: 4541380096.0000
Epoch 3/3
1/1 [==============================] - ETA: 0s - loss: 312453535629312.0000 - MSE: 312453535629312.000 - 3s 3s/step - loss: 312453535629312.0000 - MSE: 312453535629312.0000 - val_loss: 81555234816.0000 - val_MSE: 81555234816.0000


Epoch 1/3
1/1 [==============================] - 11s 11s/step - loss: 38218.7070 - MSE: 38218.7070 - val_loss: 405948.4375 - val_MSE: 405948.4375
Epoch 2/3
1/1 [==============================] - ETA: 0s - loss: 2005266304.0000 - MSE: 2005266304.000 - 10s 10s/step - loss: 2005266304.0000 - MSE: 2005266304.0000 - val_loss: 323481.9375 - val_MSE: 323481.9375
Epoch 3/3
1/1 [==============================] - ETA: 0s - loss: 102797224.0000 - MSE: 102797224.000 - 10s 10s/step - loss: 102797224.0000 - MSE: 102797224.0000 - val_loss: 271913.9062 - val_MSE: 271913.9062


Epoch 1/3
1/1 [==============================] - 12s 12s/step - loss: 36027.9102 - MSE: 36027.9102 - val_loss: 1483852.7500 - val_MSE: 1483852.7500
Epoch 2/3
1/1 [==============================] - ETA: 0s - loss: 2619430400.0000 - MSE: 2619430400.000 - 14s 14s/step - loss: 2619430400.0000 - MSE: 2619430400.0000 - val_loss: 99833.5000 - val_MSE: 99833.5000
Epoch 3/3
1/1 [==============================] - ETA: 0s - loss: 162495184.0000 - MSE: 162495184.000 - 13s 13s/step - loss: 162495184.0000 - MSE: 162495184.0000 - val_loss: 109954.7578 - val_MSE: 109954.7578


Epoch 1/3
1/1 [==============================] - 6s 6s/step - loss: 39424.8164 - MSE: 39424.8164 - val_loss: 28659.4258 - val_MSE: 28659.4258
Epoch 2/3
1/1 [==============================] - ETA: 0s - loss: 24096232.0000 - MSE: 24096232.000 - 7s 7s/step - loss: 24096232.0000 - MSE: 24096232.0000 - val_loss: 1666.4716 - val_MSE: 1666.4716
Epoch 3/3
1/1 [==============================] - ETA: 0s - loss: 1586544.3750 - MSE: 1586544.375 - 6s 6s/step - loss: 1586544.3750 - MSE: 1586544.3750 - val_loss: 10130.7744 - val_MSE: 10130.7744


Traceback (most recent call last):
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-19-f23a198db292>", line 8, in build_model
    model.add(Dense(3))
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/tensorflow/python/training/tracking/base.py", line 517, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/tensorflow/python/keras/engine/sequential.py", line 223, in add
    output_tensor = layer(self.outputs[0])
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer.py", line 951, in __call__
    return self._functional_construction_call(inputs, args, kwargs,
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer.py", l

...

Traceback (most recent call last):
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-19-f23a198db292>", line 8, in build_model
    model.add(Dense(3))
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/tensorflow/python/training/tracking/base.py", line 517, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/tensorflow/python/keras/engine/sequential.py", line 223, in add
    output_tensor = layer(self.outputs[0])
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer.py", line 951, in __call__
    return self._functional_construction_call(inputs, args, kwargs,
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer.py", l

Traceback (most recent call last):
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-19-f23a198db292>", line 8, in build_model
    model.add(Dense(3))
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/tensorflow/python/training/tracking/base.py", line 517, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/tensorflow/python/keras/engine/sequential.py", line 223, in add
    output_tensor = layer(self.outputs[0])
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer.py", line 951, in __call__
    return self._functional_construction_call(inputs, args, kwargs,
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer.py", l

Traceback (most recent call last):
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-19-f23a198db292>", line 8, in build_model
    model.add(Dense(3))
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/tensorflow/python/training/tracking/base.py", line 517, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/tensorflow/python/keras/engine/sequential.py", line 223, in add
    output_tensor = layer(self.outputs[0])
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer.py", line 951, in __call__
    return self._functional_construction_call(inputs, args, kwargs,
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer.py", l

Traceback (most recent call last):
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-19-f23a198db292>", line 8, in build_model
    model.add(Dense(3))
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/tensorflow/python/training/tracking/base.py", line 517, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/tensorflow/python/keras/engine/sequential.py", line 223, in add
    output_tensor = layer(self.outputs[0])
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer.py", line 951, in __call__
    return self._functional_construction_call(inputs, args, kwargs,
  File "/home/sepidehparhami/.conda/envs/yield/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer.py", l

RuntimeError: Too many failed attempts to build model.

In [22]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
best_hps.values

{'units': 384, 'kernel_size': 2, 'learning_rate': 0.0005}

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(X_train, y_train, epochs=10, validation_split=0.2)
eval_result = hypermodel.evaluate(X_test, y_test)
print("[test loss, test MSE]:", eval_result)